# Gated Recurrent Units (GRU)
:label:`sec_gru`

In :numref:`sec_bptt`,
we discussed how gradients are calculated
in RNNs.
In particular we found that long products of matrices can lead
to vanishing or exploding gradients.
Let us briefly think about what such
gradient anomalies mean in practice:

* We might encounter a situation where an early observation is highly
  significant for predicting all future observations. Consider the somewhat
  contrived case where the first observation contains a checksum and the goal is
  to discern whether the checksum is correct at the end of the sequence. In this
  case, the influence of the first token is vital. We would like to have some
  mechanisms for storing vital early information in a *memory cell*. Without such
  a mechanism, we will have to assign a very large gradient to this observation,
  since it affects all the subsequent observations.
* We might encounter situations where some tokens carry no pertinent
  observation. For instance, when parsing a web page there might be auxiliary
  HTML code that is irrelevant for the purpose of assessing the sentiment
  conveyed on the page. We would like to have some mechanism for *skipping* such
  tokens in the latent state representation.
* We might encounter situations where there is a logical break between parts of
  a sequence. For instance, there might be a transition between chapters in a
  book, or a transition between a bear and a bull market for securities. In
  this case it would be nice to have a means of *resetting* our internal state
  representation.

A number of methods have been proposed to address this. One of the earliest is long short-term memory :cite:`Hochreiter.Schmidhuber.1997` which we
will discuss in :numref:`sec_lstm`. The gated recurrent unit (GRU)
:cite:`Cho.Van-Merrienboer.Bahdanau.ea.2014` is a slightly more streamlined
variant that often offers comparable performance and is significantly faster to
compute  :cite:`Chung.Gulcehre.Cho.ea.2014`.
Due to its simplicity, let us start with the GRU.

## Gated Hidden State

The key distinction between vanilla RNNs and GRUs
is that the latter support gating of the hidden state.
This means that we have dedicated mechanisms for
when a hidden state should be *updated* and
also when it should be *reset*.
These mechanisms are learned and they address the concerns listed above.
For instance, if the first token is of great importance
we will learn not to update the hidden state after the first observation.
Likewise, we will learn to skip irrelevant temporary observations.
Last, we will learn to reset the latent state whenever needed.
We discuss this in detail below.


### Reset Gate and Update Gate

The first thing we need to introduce are
the *reset gate* and the *update gate*.
We engineer them to be vectors with entries in $(0, 1)$
such that we can perform convex combinations.
For instance,
a reset gate would allow us to control how much of the previous state we might still want to remember.
Likewise, an update gate would allow us to control how much of the new state is just a copy of the old state.

We begin by engineering these gates.
:numref:`fig_gru_1` illustrates the inputs for both
the reset and update gates in a GRU, given the input
of the current time step
and the hidden state of the previous time step.
The outputs of two gates
are given by two fully-connected layers
with a sigmoid activation function.

![Computing the reset gate and the update gate in a GRU model.](https://raw.githubusercontent.com/d2l-ai/d2l-en/master/img/gru-1.svg)
:label:`fig_gru_1`

Mathematically,
for a given time step $t$,
suppose that the input is
a minibatch
$\mathbf{X}_t \in \mathbb{R}^{n \times d}$ (number of examples: $n$, number of inputs: $d$) and the hidden state of the previous time step is $\mathbf{H}_{t-1} \in \mathbb{R}^{n \times h}$ (number of hidden units: $h$). Then, the reset gate $\mathbf{R}_t \in \mathbb{R}^{n \times h}$ and update gate $\mathbf{Z}_t \in \mathbb{R}^{n \times h}$ are computed as follows:

$$
\begin{aligned}
\mathbf{R}_t = \sigma(\mathbf{X}_t \mathbf{W}_{xr} + \mathbf{H}_{t-1} \mathbf{W}_{hr} + \mathbf{b}_r),\\
\mathbf{Z}_t = \sigma(\mathbf{X}_t \mathbf{W}_{xz} + \mathbf{H}_{t-1} \mathbf{W}_{hz} + \mathbf{b}_z),
\end{aligned}
$$

where $\mathbf{W}_{xr}, \mathbf{W}_{xz} \in \mathbb{R}^{d \times h}$ and
$\mathbf{W}_{hr}, \mathbf{W}_{hz} \in \mathbb{R}^{h \times h}$ are weight
parameters and $\mathbf{b}_r, \mathbf{b}_z \in \mathbb{R}^{1 \times h}$ are
biases. 
Note that broadcasting (see :numref:`subsec_broadcasting`) is triggered during the summation.
We use sigmoid functions (as introduced in :numref:`sec_mlp`) to transform input values to the interval $(0, 1)$.

### Candidate Hidden State

Next, let us
integrate the reset gate $\mathbf{R}_t$ with
the regular latent state updating mechanism
in :eqref:`rnn_h_with_state`.
It leads to the following
*candidate hidden state*
$\tilde{\mathbf{H}}_t \in \mathbb{R}^{n \times h}$ at time step $t$:

$$\tilde{\mathbf{H}}_t = \tanh(\mathbf{X}_t \mathbf{W}_{xh} + \left(\mathbf{R}_t \odot \mathbf{H}_{t-1}\right) \mathbf{W}_{hh} + \mathbf{b}_h),$$
:eqlabel:`gru_tilde_H`

where $\mathbf{W}_{xh} \in \mathbb{R}^{d \times h}$ and $\mathbf{W}_{hh} \in \mathbb{R}^{h \times h}$
are weight parameters,
$\mathbf{b}_h \in \mathbb{R}^{1 \times h}$
is the bias,
and the symbol $\odot$ is the Hadamard (elementwise) product operator.
Here we use a nonlinearity in the form of tanh to ensure that the values in the candidate hidden state remain in the interval $(-1, 1)$.

The result is a *candidate* since we still need to incorporate the action of the update gate.
Comparing with :eqref:`rnn_h_with_state`,
now the influence of the previous states
can be reduced with the
elementwise multiplication of
$\mathbf{R}_t$ and $\mathbf{H}_{t-1}$
in :eqref:`gru_tilde_H`.
Whenever the entries in the reset gate $\mathbf{R}_t$ are close to 1, we recover a vanilla RNN such as in :eqref:`rnn_h_with_state`.
For all entries of the reset gate $\mathbf{R}_t$ that are close to 0, the candidate hidden state is the result of an MLP with $\mathbf{X}_t$ as the input. Any pre-existing hidden state is thus *reset* to defaults.

:numref:`fig_gru_2` illustrates the computational flow after applying the reset gate.

![Computing the candidate hidden state in a GRU model.](https://raw.githubusercontent.com/d2l-ai/d2l-en/master/img/gru-2.svg)
:label:`fig_gru_2`


### Hidden State

Finally, we need to incorporate the effect of the update gate $\mathbf{Z}_t$. This determines the extent to which the new hidden state $\mathbf{H}_t \in \mathbb{R}^{n \times h}$ is just the old state $\mathbf{H}_{t-1}$ and by how much the new candidate state $\tilde{\mathbf{H}}_t$ is used.
The update gate $\mathbf{Z}_t$ can be used for this purpose, simply by taking elementwise convex combinations between both $\mathbf{H}_{t-1}$ and $\tilde{\mathbf{H}}_t$.
This leads to the final update equation for the GRU:

$$\mathbf{H}_t = \mathbf{Z}_t \odot \mathbf{H}_{t-1}  + (1 - \mathbf{Z}_t) \odot \tilde{\mathbf{H}}_t.$$


Whenever the update gate $\mathbf{Z}_t$ is close to 1, we simply retain the old state. In this case the information from $\mathbf{X}_t$ is essentially ignored, effectively skipping time step $t$ in the dependency chain. In contrast, whenever $\mathbf{Z}_t$ is close to 0, the new latent state $\mathbf{H}_t$ approaches the candidate latent state $\tilde{\mathbf{H}}_t$. These designs can help us cope with the vanishing gradient problem in RNNs and better capture dependencies for sequences with large time step distances.
For instance,
if the update gate has been close to 1
for all the time steps of an entire subsequence,
the old hidden state at the time step of its beginning
will be easily retained and passed
to its end,
regardless of the length of the subsequence.



:numref:`fig_gru_3` illustrates the computational flow after the update gate is in action.

![Computing the hidden state in a GRU model.](https://raw.githubusercontent.com/d2l-ai/d2l-en/master/img/gru-3.svg)
:label:`fig_gru_3`


In summary, GRUs have the following two distinguishing features:

* Reset gates help capture short-term dependencies in sequences.
* Update gates help capture long-term dependencies in sequences.

## Implementation from Scratch

To gain a better understanding of the GRU model, let us implement it from scratch. We begin by reading the time machine dataset that we used in :numref:`sec_rnn_scratch`. The code for reading the dataset is given below.


In [1]:
%use @file[../djl.json]
%use lets-plot
@file:DependsOn("../D2J-1.0-SNAPSHOT.jar")
import jp.live.ugai.d2j.timemachine.RNNModelScratch
import jp.live.ugai.d2j.timemachine.TimeMachine.trainCh8
import jp.live.ugai.d2j.timemachine.TimeMachineDataset
import jp.live.ugai.d2j.timemachine.Vocab
import jp.live.ugai.d2j.RNNModel
import jp.live.ugai.d2j.StopWatch
import jp.live.ugai.d2j.Accumulator
import jp.live.ugai.d2j.Training

// %load ../utils/djl-imports
// %load ../utils/plot-utils
// %load ../utils/Functions.java
// %load ../utils/Functions.java
// %load ../utils/PlotUtils.java

// %load ../utils/StopWatch.java
// %load ../utils/Accumulator.java
// %load ../utils/Animator.java
// %load ../utils/Training.java
// %load ../utils/timemachine/Vocab.java
// %load ../utils/timemachine/RNNModel.java
// %load ../utils/timemachine/RNNModelScratch.java
// %load ../utils/timemachine/TimeMachine.java
// %load ../utils/timemachine/TimeMachineDataset.java
import kotlin.random.Random
import kotlin.collections.List
import kotlin.collections.Map
import kotlin.Pair

In [2]:
val manager = NDManager.newBaseManager()

In [3]:
val batchSize = 32
val numSteps = 35

val dataset = TimeMachineDataset.Builder()
        .setManager(manager)
        .setMaxTokens(10000)
        .setSampling(batchSize, false)
        .setSteps(numSteps)
        .build()
dataset.prepare()
val vocab = dataset.vocab

UniqTokens: 29


### Initializing Model Parameters

The next step is to initialize the model parameters.
We draw the weights from a Gaussian distribution
with standard deviation to be 0.01 and set the bias to 0. The hyperparameter `num_hiddens` defines the number of hidden units.
We instantiate all weights and biases relating to the update gate, the reset gate, the candidate hidden state,
and the output layer.

In [4]:
    fun normal(shape: Shape, device: Device): NDArray {
        return manager.randomNormal(0.0f, 0.01f, shape, DataType.FLOAT32, device)
    }

    fun three(numInputs: Int, numHiddens: Int, device: Device): NDList {
        return NDList(
            normal(Shape(numInputs.toLong(), numHiddens.toLong()), device),
            normal(Shape(numHiddens.toLong(), numHiddens.toLong()), device),
            manager.zeros(Shape(numHiddens.toLong()), DataType.FLOAT32, device)
        )
    }

    fun getParams(vocabSize: Int, numHiddens: Int, device: Device): NDList {
        // Update gate parameters
        var temp = three(vocabSize, numHiddens, device)
        val W_xz = temp[0]
        val W_hz = temp[1]
        val b_z = temp[2]

        // Reset gate parameters
        temp = three(vocabSize, numHiddens, device)
        val W_xr = temp[0]
        val W_hr = temp[1]
        val b_r = temp[2]

        // Candidate hidden state parameters
        temp = three(vocabSize, numHiddens, device)
        val W_xh = temp[0]
        val W_hh = temp[1]
        val b_h = temp[2]

        // Output layer parameters
        val W_hq = normal(Shape(numHiddens.toLong(), vocabSize.toLong()), device)
        val b_q: NDArray = manager.zeros(Shape(vocabSize.toLong()), DataType.FLOAT32, device)

        // Attach gradients
        val params = NDList(W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q)
        for (param in params) {
            param.setRequiresGradient(true)
        }
        return params
    }

### Defining the Model

Now we will define the hidden state initialization function `init_gru_state`. Just like the `init_rnn_state` function defined in :numref:`sec_rnn_scratch`, this function returns a tensor with a shape (batch size, number of hidden units) whose values are all zeros.


In [5]:
    fun initGruState(batchSize: Int, numHiddens: Int, device: Device): NDList {
        return NDList(manager.zeros(Shape(batchSize.toLong(), numHiddens.toLong()), DataType.FLOAT32, device))
    }

Now we are ready to define the GRU model.
Its structure is the same as that of the basic RNN cell, except that the update equations are more complex.


In [6]:
    fun gru(inputs: NDArray, state: NDList, params: NDList): Pair<NDArray, NDList> {
        val W_xz = params[0]
        val W_hz = params[1]
        val b_z = params[2]
        val W_xr = params[3]
        val W_hr = params[4]
        val b_r = params[5]
        val W_xh = params[6]
        val W_hh = params[7]
        val b_h = params[8]
        val W_hq = params[9]
        val b_q = params[10]
        var H = state[0]
        val outputs = NDList()
        var X: NDArray
        var Y: NDArray
        var Z: NDArray
        var R: NDArray
        var H_tilda: NDArray
        for (i in 0 until inputs.size(0)) {
            X = inputs[i]
            Z = Activation.sigmoid(X.dot(W_xz).add(H.dot(W_hz).add(b_z)))
            R = Activation.sigmoid(X.dot(W_xr).add(H.dot(W_hr).add(b_r)))
            H_tilda = Activation.tanh(X.dot(W_xh).add(R.mul(H).dot(W_hh).add(b_h)))
            H = Z.mul(H).add(Z.mul(-1).add(1).mul(H_tilda))
            Y = H.dot(W_hq).add(b_q)
            outputs.add(Y)
        }
        return Pair(if (outputs.size > 1) NDArrays.concat(outputs) else outputs[0], NDList(H))
    }

### Training and Prediction

Training and prediction work in exactly the same manner as in :numref:`sec_rnn_scratch`.
After training,
we print out the perplexity on the training set
and the predicted sequence following
the provided prefixes "time traveller" and "traveller", respectively.


In [10]:
    val vocabSize = vocab.length()
    val numHiddens = 256
    val device = manager.device
    val numEpochs = Integer.getInteger("MAX_EPOCH", 500)

    val lr = 1
    val model = RNNModelScratch(vocabSize, numHiddens, device, ::getParams, ::initGruState, ::gru)
    trainCh8(model, dataset, vocab, lr, numEpochs, device, false, manager)

10 : 17.271841167371946
20 : 15.558340521802963
30 : 12.932673074172952
40 : 11.517789793633652
50 : 10.859578228883809
60 : 10.396758730356678
70 : 10.022118791155375
80 : 9.672249909703204
90 : 9.318414060908909
100 : 8.975387667401227
110 : 8.65112473599814
120 : 8.357729672118614
130 : 8.083047618563453
140 : 7.802219653388491
150 : 7.5133214511495
160 : 7.332238049111319
170 : 6.954259703723563
180 : 6.6869890462785415
190 : 6.544543125433988
200 : 6.182894323830925
210 : 5.889348634516324
220 : 5.567831734215796
230 : 5.2998297178611535
240 : 4.866570091210992
250 : 4.444854192998585
260 : 4.0194286519449225
270 : 3.744644844403568
280 : 3.3098190775001353
290 : 2.8249033869116666
300 : 2.6687337604462646
310 : 2.1520669592613304
320 : 1.929144613811204
330 : 1.5491798151163925
340 : 1.5598138835102993
350 : 1.5115303479780329
360 : 1.1494822851989233
370 : 1.156347623632971
380 : 1.0861530997355475
390 : 1.2570793742403263
400 : 1.0623924293646037
410 : 1.0486809975994094
420 : 

## Concise Implementation

In high-level APIs,
we can directly
instantiate a GPU model.
This encapsulates all the configuration detail that we made explicit above.
The code is significantly faster as it uses compiled operators rather than Python for many details that we spelled out before.


In [11]:
    val gruLayer = GRU.builder()
        .setNumLayers(1)
        .setStateSize(numHiddens)
        .optReturnState(true)
        .optBatchFirst(false)
        .build()
    val modelConcise = RNNModel(gruLayer, vocab.length())
    trainCh8(modelConcise, dataset, vocab, lr, numEpochs, device, false, manager)

10 : 17.2957388424852
20 : 15.612232199921735
30 : 12.95745128904966
40 : 11.511394359966346
50 : 10.831249084171018
60 : 10.35717830890521
70 : 9.973543260723103
80 : 9.612699276279152
90 : 9.265637751238879
100 : 8.945641845277848
110 : 8.647103269097705
120 : 8.356512771273225
130 : 8.094383229732669
140 : 7.832642282375142
150 : 7.573023341670194
160 : 7.303130181751629
170 : 7.073852246195889
180 : 6.777475831054104
190 : 6.590961527373202
200 : 6.279959571682691
210 : 5.953709416281121
220 : 5.71520063433334
230 : 5.399517493219079
240 : 5.152538053090552
250 : 4.820729907284465
260 : 4.452173965108438
270 : 4.0641103697376195
280 : 3.7981879592290517
290 : 3.3607761221456802
300 : 3.009786339446698
310 : 2.714685723469379
320 : 2.2613384639440692
330 : 1.9874101788284821
340 : 1.784293217582859
350 : 1.4896686084430264
360 : 1.3501950621358667
370 : 1.4154513535272335
380 : 1.1388849442235562
390 : 1.2025281479317587
400 : 1.0835807950734486
410 : 1.0702961199199543
420 : 1.0684

## Summary

* Gated RNNs can better capture dependencies for sequences with large time step distances.
* Reset gates help capture short-term dependencies in sequences.
* Update gates help capture long-term dependencies in sequences.
* GRUs contain basic RNNs as their extreme case whenever the reset gate is switched on. They can also skip subsequences by turning on the update gate.


## Exercises

1. Assume that we only want to use the input at time step $t'$ to predict the output at time step $t > t'$. What are the best values for the reset and update gates for each time step?
1. Adjust the hyperparameters and analyze the their influence on running time, perplexity, and the output sequence.
1. Compare runtime, perplexity, and the output strings for `rnn.RNN` and `rnn.GRU` implementations with each other.
1. What happens if you implement only parts of a GRU, e.g., with only a reset gate or only an update gate?
